In [1]:
# Standard Library
import uuid
import argparse

# Third Party Library
import pandas as pd
from ex_utils.config.dataset import dataset_names
from ex_utils.share import (
    ex_path,
    generate_seed_median_df,
)

n = 100

n_sample = 50
n_split = 10

d_names = [
    "1138_bus",
    "USpowerGrid",
    "dwt_1005",
    "poli",
    "qh882",
    "3elt",
    "dwt_2680",
]


df_paths = [
    ex_path.joinpath(
        f"data/grid/qh882/n_split={n_split}/seed={data_seed}.pkl"
    )
    for data_seed in list(range(15))
]
df = pd.concat([pd.read_pickle(df_path) for df_path in df_paths])
mdf = generate_seed_median_df(df)
for _ in range(n):
    sampled_grid = mdf.sample(n=n_sample)

    id = uuid.uuid4()
    for d_name in d_names:
        df_paths = [
            ex_path.joinpath(
                f"data/grid/{d_name}/n_split={n_split}/seed={data_seed}.pkl"
            )
            for data_seed in list(range(15))
        ]
        df = pd.concat([pd.read_pickle(df_path) for df_path in df_paths])
        mdf = generate_seed_median_df(df)
        export_path = ex_path.joinpath(
            f"data/sampled_points/{d_name}/n_split={n_split}/n_sample={n_sample}/{id}.pkl"
        )

        export_path.parent.mkdir(parents=True, exist_ok=True)
        d_sampled = mdf.loc[sampled_grid.index]
        d_sampled.to_pickle(export_path)